In [ ]:
import pandas as pd
import numpy as np
import datetime
from functools import reduce

data = pd.read_csv("../Resources/sars_cases.csv")
death = pd.read_csv("../Resources/sars_deaths.csv")
hospitalization = pd.read_csv("../Resources/sars_hospitalizations.csv")
population = pd.read_csv("population.csv")

#adding suffix
hospitalization = hospitalization.add_suffix("_hospitalization")
#census = pd.read_csv("../Resources/Census_Wash_Counties_Income_Poverty.csv")

cleandata = pd.DataFrame.copy(data)

day = pd.Series(cleandata["WeekStartDate"]).str.split("/").str[1]
month = pd.Series(cleandata["WeekStartDate"]).str.split("/").str[0]
year = pd.Series(cleandata["WeekStartDate"]).str.split("/").str[2]

month = month.map(lambda x: x.zfill(2))
day = day.map(lambda x: x.zfill(2))


cleandata["WeekStartDate"] = year + "-" + month + "-" + day




dataframes = [cleandata, death, hospitalization]

#mergedf = reduce(lambda  left,right: pd.merge(left,right,on=['County', "WeekStartDate"], suffixes= ("_cases", "_death", "_hospitalization"), how='outer'), dataframes)

mergedf = pd.merge(cleandata, death, on = ["County", "WeekStartDate"], suffixes = ("_cases", "_death"), how= "outer", indicator = True)#.fillna(0)
#mergedf[mergedf["_merge"] == "both"]

merge1 = pd.DataFrame(mergedf)

mergedf2 = pd.merge(merge1, hospitalization, left_on = ["County", "WeekStartDate"], right_on = ["County_hospitalization", "WeekStartDate_hospitalization"], how = "outer", indicator = "exists")

#mergedf.fillna(0)

mergedf2.to_csv("MergeWeek.csv")


In [ ]:
mergedf3 = mergedf2
mergedf3["Month"] = mergedf3["WeekStartDate"].str.split("-").str[1]

mergemonth = mergedf3.groupby(["County", "Month"])
mergemonth = mergemonth[['NewPos_All', 'Age 0-19_cases',
       'Age 20-39_cases', 'Age 40-59_cases', 'Age 60-79_cases',
       'Age 80+_cases', 'Positive UnkAge_cases', 'dtm_updated_cases', 'Deaths',
       'Age 0-19_death', 'Age 20-39_death', 'Age 40-59_death',
       'Age 60-79_death', 'Age 80+_death', 'Positive UnkAge_death',
       'dtm_updated_death', 'County_hospitalization',
       'Hospitalizations_hospitalization',
       'Age 0-19_hospitalization', 'Age 20-39_hospitalization',
       'Age 40-59_hospitalization', 'Age 60-79_hospitalization',
       'Age 80+_hospitalization', 'Positive UnkAge_hospitalization',
       'dtm_updated_hospitalization']].sum()
mergemonth.to_csv("MergeMonth.csv")



In [ ]:
census = pd.read_csv("../census_politics_merge_mike2.csv")

mergedf4 = mergemonth.reset_index()
mergedf5 = pd.merge(mergedf4, census, on = "County", how="outer")
mergedf5 = mergedf5.drop(columns=['Counter'])
#mergedf5.to_csv("Final Merge 7_15_20")


In [ ]:
county_gb = mergedf4.groupby("County")
county_sum = county_gb.sum()
county_sum
merge_census = pd.merge(county_sum, census, on = "County", how="outer")
covid_reg = pd.merge(merge_census, population, on = "County", how="outer")
covid_reg = covid_reg.drop([31])
covid_reg = covid_reg.drop(columns=['Counter'])
covid_reg["Pop_per100k"] = covid_reg["Population"]/100000
covid_reg = covid_reg.fillna(0)
covid_reg["Case_per100k"] = covid_reg["NewPos_All"]/covid_reg["Pop_per100k"]
covid_reg["Deaths_per100k"] = covid_reg["Deaths"]/covid_reg["Pop_per100k"]
covid_reg["Hoptializations_per100k"] = covid_reg["Hospitalizations_hospitalization"]/covid_reg["Pop_per100k"]
covid_reg.to_csv("Final Merge 7_16_20.csv")
covid_reg